1. CSP로 이진분류
2. 머신러닝 기법 이용(로지스틱) 데이터 분류
3. 전처리된 데이터를 전결합층 fully connected 엡실러 층3개로 해서 해보기 

In [4]:
!pip install numpy moabb

In [5]:
from moabb.datasets import BNCI2014_008
from moabb.paradigms import P300
import pandas as pd

In [6]:
# 필수 라이브러리 불러오기
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from mne.decoding import CSP
from moabb.datasets import BNCI2014_008
from moabb.paradigms import P300
from sklearn.linear_model import LogisticRegression

In [7]:
dataset = BNCI2014_008()

In [8]:
paradigm = P300()

In [9]:
X, labels, meta = paradigm.get_data(dataset=dataset)

C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 4200 events (all good), 0 – 1 s (baseline off), ~65.9 MiB, data loaded,
 'Target': 700
 'NonTarget': 3500>
  warn(f"warnEpochs {epochs}")
C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 4200 events (all good), 0 – 1 s (baseline off), ~65.9 MiB, data loaded,
 'Target': 700
 'NonTarget': 3500>
  warn(f"warnEpochs {epochs}")
C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate a

In [10]:
print("X shape:", X.shape)  # (33600, 8, 257) -> (샘플 수, 채널 수, 시간 축)
print("Labels shape:", labels.shape)  # (33600,)

X shape: (33600, 8, 257)
Labels shape: (33600,)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [12]:
print("X shape:", X_train.shape)  # (33600, 8, 257) -> (샘플 수, 채널 수, 시간 축)
print("Labels shape:", y_train.shape)  # (33600,)

X shape: (26880, 8, 257)
Labels shape: (26880,)


In [13]:
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

# CSP 변환 수행 (3D 형태 유지)
X_train_csp = csp.fit_transform(X_train, y_train)
X_test_csp = csp.transform(X_test)

Computing rank from data with rank=None
    Using tolerance 1e+02 (2.2e-16 eps * 8 dim * 5.7e+16  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=NonTarget covariance using EMPIRICAL
Done.
Estimating class=Target covariance using EMPIRICAL
Done.


In [14]:
X_train_csp.shape

(26880, 4)

In [15]:
log_reg = LogisticRegression(max_iter=1000)

# 모델 학습
log_reg.fit(X_train_csp, y_train)

# 예측 수행
y_pred_train_log = log_reg.predict(X_train_csp)
y_pred_test_log = log_reg.predict(X_test_csp)

In [17]:
# 정확도 출력
train_accuracy = accuracy_score(y_train, y_pred_train_log)
test_accuracy = accuracy_score(y_test, y_pred_test_log)
print(f"🔹 로지스틱 회귀 Train 정확도: {train_accuracy:.4f}")
print(f"🔹 로지스틱 회귀 Test 정확도: {test_accuracy:.4f}")

🔹 로지스틱 회귀 Train 정확도: 0.8325
🔹 로지스틱 회귀 Test 정확도: 0.8366
